In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from synorchestrator import orchestrator
from synorchestrator import config

In [3]:
config.show()


Orchestrator options:

Evaluation Queues
---------------------------------------------------------------------------
wflow0: github.com/dockstore-testing/md5sum-checker [CWL]
wflow1: github.com/dockstore-testing/md5sum-checker/wdl [WDL]
wflow2: github.com/DataBiosphere/topmed-workflows/TopMed_Variant_Caller [WDL]
wflow3: github.com/DataBiosphere/topmed-workflows/u_of_Michigan_alignment_pipeline [WDL]

Tool Registries
---------------------------------------------------------------------------
dockstore: dockstore.org:8443

Workflow Services
---------------------------------------------------------------------------
hca-cromwell: g0n2qjnu94.execute-api.us-east-1.amazonaws.com/test
broad-cromwell: 35.226.102.121:9090
arvados-wes: wes.qr1hi.arvadosapi.com
local: 0.0.0.0:8080


In [4]:
submissions = orchestrator.run_all({
    'wflow0': ['arvados-wes', 'broad-cromwell'],
    'wflow2': ['hca-cromwell', 'broad-cromwell'],
    'wflow3': ['hca-cromwell', 'broad-cromwell']
})

INFO:synorchestrator.orchestrator:Preparing checker workflow run request for 'github.com/DataBiosphere/topmed-workflows/u_of_Michigan_alignment_pipeline' from  'dockstore''
INFO:synorchestrator.eval:Created new job submission:
 - evaluation queue: wflow3 (github.com/DataBiosphere/topmed-workflows/u_of_Michigan_alignment_pipeline)
 - submission ID: '290529211034671585'
 - data:
{
  "workflow_descriptor": "import \"https://raw.githubusercontent.com/DataBiosphere/topmed-workflows/1.13.0/aligner/u_of_michigan_aligner/u_of_michigan_aligner.wdl\" as TopMed_aligner\nimport \"https://raw.githubusercontent.com/DataBiosphere/topmed-workflows/1.13.0/aligner/functional-equivalence-checker/topmed-alignment-checker.wdl\" as checker\n\nworkflow checkerWorkflow {\n  Int expectedNumofReads\n  String docker_image\n\n  File input_crai_file\n  File input_cram_file\n\n  File ref_alt\n  File ref_bwt\n  File ref_sa\n  File ref_amb\n  File ref_ann\n  File ref_pac\n\n  File ref_fasta\n  File ref_fasta_index\n\

INFO:synorchestrator.orchestrator:Preparing checker workflow run request for 'github.com/DataBiosphere/topmed-workflows/TopMed_Variant_Caller' from  'dockstore''
INFO:synorchestrator.eval:Created new job submission:
 - evaluation queue: wflow2 (github.com/DataBiosphere/topmed-workflows/TopMed_Variant_Caller)
 - submission ID: '290529211040766509'
 - data:
{
  "workflow_descriptor": "import \"https://raw.githubusercontent.com/DataBiosphere/topmed-workflows/1.11.0/variant-caller/variant-caller-wdl/topmed_freeze3_calling.wdl\" as TopMed_variantcaller\nimport \"https://raw.githubusercontent.com/DataBiosphere/topmed-workflows/1.11.0/variant-caller/variant-caller-wdl-checker/topmed-variantcaller-checker.wdl\" as checker\n\nworkflow checkerWorkflow {\n  File inputTruthVCFFile\n\n  # Deprecated: No need to input this anymore\n  # Disk size requirements will be calculated internally\n  # This will be removed in the next release\n  Float? reference_files_size\n\n  String docker_image\n\n  Array[

INFO:synorchestrator.orchestrator:Preparing checker workflow run request for 'github.com/dockstore-testing/md5sum-checker' from  'dockstore''
INFO:synorchestrator.eval:Created new job submission:
 - evaluation queue: wflow0 (github.com/dockstore-testing/md5sum-checker)
 - submission ID: '290529211044953971'
 - data:
{
  "workflow_descriptor": "{\n    \"cwlVersion\": \"v1.0\", \n    \"$graph\": [\n        {\n            \"class\": \"Workflow\", \n            \"requirements\": [\n                {\n                    \"class\": \"SubworkflowFeatureRequirement\"\n                }\n            ], \n            \"inputs\": [\n                {\n                    \"type\": \"string\", \n                    \"id\": \"#main/expected_md5\"\n                }, \n                {\n                    \"type\": \"File\", \n                    \"id\": \"#main/input_file\"\n                }\n            ], \n            \"outputs\": [\n                {\n                    \"type\": \"File\",

INFO:synorchestrator.eval:Created new job submission:
 - evaluation queue: wflow0 (github.com/dockstore-testing/md5sum-checker)
 - submission ID: '290529211048309644'
 - data:
{
  "workflow_descriptor": "{\n    \"cwlVersion\": \"v1.0\", \n    \"$graph\": [\n        {\n            \"class\": \"Workflow\", \n            \"requirements\": [\n                {\n                    \"class\": \"SubworkflowFeatureRequirement\"\n                }\n            ], \n            \"inputs\": [\n                {\n                    \"type\": \"string\", \n                    \"id\": \"#main/expected_md5\"\n                }, \n                {\n                    \"type\": \"File\", \n                    \"id\": \"#main/input_file\"\n                }\n            ], \n            \"outputs\": [\n                {\n                    \"type\": \"File\", \n                    \"outputSource\": \"#main/checker/results_file\", \n                    \"id\": \"#main/workflow_output_file\"\n       

INFO:synorchestrator.orchestrator:Submitting job '290529211040766509' for eval 'wflow2' to WES endpoint 'broad-cromwell'
INFO:synorchestrator.orchestrator:Submitting job '290529211038681861' for eval 'wflow2' to WES endpoint 'hca-cromwell'
INFO:synorchestrator.orchestrator:Submitting job '290529211048309644' for eval 'wflow0' to WES endpoint 'broad-cromwell'
INFO:synorchestrator.orchestrator:Submitting job '290529211044953971' for eval 'wflow0' to WES endpoint 'arvados-wes'


In [5]:
orchestrator.monitor(submissions)

submission_status elapsed_time      job  \
TopMed_Variant_Caller            290529211038681861         SUBMITTED    0h:1m:12s  checker   
                                 290529211040766509         SUBMITTED   0h:20m:20s  checker   
md5sum-checker                   290529211044953971         SUBMITTED    0h:0m:32s  checker   
                                 290529211048309644         SUBMITTED    0h:0m:22s  checker   
u_of_Michigan_alignment_pipeline 290529211034671585         SUBMITTED    0h:1m:22s  checker   
                                 290529211036592693         SUBMITTED   0h:27m:39s  checker   

                                                             wes_id  \
TopMed_Variant_Caller            290529211038681861    hca-cromwell   
                                 290529211040766509  broad-cromwell   
md5sum-checker                   290529211044953971     arvados-wes   
                                 290529211048309644  broad-cromwell   
u_of_Michigan_alignment_pipeline 290529211034671585    hca-cromwell   
                                 290529211036592693  broad-cromwell   

                                                                             queue_id  \
TopMed_Variant_Caller            290529211038681861             TopMed_Variant_Caller   
                                 290529211040766509             TopMed_Variant_Caller   
md5sum-checker                   290529211044953971                    md5sum-checker   
                                 290529211048309644                    md5sum-checker   
u_of_Michigan_alignment_pipeline 290529211034671585  u_of_Michigan_alignment_pipeline   
                                 290529211036592693  u_of_Michigan_alignment_pipeline   

                                                         run_status  \
TopMed_Variant_Caller            290529211038681861        COMPLETE   
                                 290529211040766509        COMPLETE   
md5sum-checker                   290529211044953971        COMPLETE   
                                 290529211048309644  EXECUTOR_ERROR   
u_of_Michigan_alignment_pipeline 290529211034671585        COMPLETE   
                                 290529211036592693        COMPLETE   

                                                                                   run_id  \
TopMed_Variant_Caller            290529211038681861  824d644b-f776-4392-aa96-2bd27016fbc4   
                                 290529211040766509  c4732d4e-33df-4b8d-bed1-645dc8239af3   
md5sum-checker                   290529211044953971           qr1hi-xvhdp-as5qntn3l0eqfxq   
                                 290529211048309644  aa5eb354-93ec-4685-8bd2-4e9fb5f99dd4   
u_of_Michigan_alignment_pipeline 290529211034671585  3ab634d4-f952-4809-9913-e6948d573bbd   
                                 290529211036592693  9d1c69a3-952e-47bb-9f0e-ef9cc4d6f46f   

                                                                   start_time  
TopMed_Variant_Caller            290529211038681861  Tue May 29 21:10:50 2018  
                                 290529211040766509  Tue May 29 21:10:50 2018  
md5sum-checker                   290529211044953971  Tue May 29 21:10:51 2018  
                                 290529211048309644  Tue May 29 21:10:50 2018  
u_of_Michigan_alignment_pipeline 290529211034671585  Tue May 29 21:10:49 2018  
                                 290529211036592693  Tue May 29 21:10:49 2018

Done


In [7]:
from synorchestrator.wes.client import WESClient
broadclient = WESClient(**config.wes_config['broad-cromwell'])

In [8]:
workflow_run = broadclient.get_workflow_run('aa5eb354-93ec-4685-8bd2-4e9fb5f99dd4')

In [12]:
workflow_run['request']['workflow_params']

{u'expected_md5': u'00579a00e3e7fa0674428ac7049423e2',
 u'input_file': {u'class': u'File',
  u'location': u'https://raw.githubusercontent.com/dockstore-testing/md5sum-checker/develop/md5sum.input'}}